In [4]:
## Libraries and setup
# !pip install diagonal_b6
import diagonal_b6 as b6
import os
import json
import geopandas as gpd
import pandas as pd
import shapely

# get working directory
working_directory =  os.getcwd()

# data directory
data_directory = working_directory.replace('Scripts', 'Data')

# connect to the Bedrock engine
w = b6.connect_insecure("cdtweek-2023.diagonal.works:8002")

In [5]:
## get lsoas
# get Bedrock data for Liverpool study area
liverpool = b6.cap_polygon(b6.ll(53.4127,-2.9608), 10000)

# get lsoas
lsoas = b6.find(b6.and_(b6.tagged("#boundary", "lsoa"), b6.intersecting(liverpool)))

In [6]:
# test = b6.find(b6.and_("#building", b6.intersecting(liverpool)))

#  # export to geojson
# buildings_geojson = w(b6.to_geojson_collection(test))
    
#     # save geojson to gdf
# buildings_gdf = gpd.GeoDataFrame.from_features(buildings_geojson)
    
#     # set CRS
# buildings_gdf = buildings_gdf.set_crs("EPSG:4326")

In [7]:
## get buildings

# create a list of building types
building_types = {'attribute': ["#building", "#building", "#building",  "#building", "#building", "#building", '#building'],
                  'type': ['bungalow','apartments', 'semidetached_house', 'detached', 'terrace','residential', 'house']}

building_types_df = pd.DataFrame(building_types)

# create an empty list to store the dataframes
combined_gdf = []

# loop through every building type to get all buildings without crashing the request
for i in range(len(building_types_df)):
    
    # extract the current building type and query condition
    current_building_type = building_types_df.loc[i, 'type']
    current_condition = (b6.tagged(building_types_df.loc[i, 'attribute'], current_building_type))
    
    # find the buildings that match the current building type
    buildings = b6.find(b6.and_(current_condition, b6.intersecting(liverpool)))
    
    # export to geojson
    buildings_geojson = w(b6.to_geojson_collection(buildings))
    
    # save geojson to gdf
    buildings_gdf = gpd.GeoDataFrame.from_features(buildings_geojson)
    
    # set CRS
    buildings_gdf = buildings_gdf.set_crs("EPSG:4326")
    
    # Calculate the centroid of each geometry
    buildings_gdf["centre"] = buildings_gdf["geometry"].centroid
    
    # Convert the "centre" column to a tuple of x, y coordinates and save to list
    buildings_gdf["centre"] = buildings_gdf["centre"].apply(lambda p: (p.x, p.y))
    
    # append the current buildings_gdf to the list
    combined_gdf.append(buildings_gdf)

# combine all the dataframes in the list into one dataframe
combined_gdf = pd.concat(combined_gdf)


C:\Users\b8008458\AppData\Local\Temp\ipykernel_20676\1668171591.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_gdf["centre"] = buildings_gdf["geometry"].centroid
C:\Users\b8008458\AppData\Local\Temp\ipykernel_20676\1668171591.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_gdf["centre"] = buildings_gdf["geometry"].centroid
C:\Users\b8008458\AppData\Local\Temp\ipykernel_20676\1668171591.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_gdf["centre"] = buildings_gdf["geometry"].centroid
C:\Users\b8008458\AppData\Local\Temp\ipyker

In [8]:
buildings_gdf = combined_gdf

In [9]:
# Create a unique ID for every building in the gdf
buildings_gdf['buildings_id'] = range(len(buildings_gdf))


## convert centre coordinates to a list of pairs
building_centre_coords = [tuple(map(float, str(x).strip('()').split(', '))) for x in buildings_gdf['centre']]

c:\Users\b8008458\Anaconda3\envs\clean_ox\Lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


In [10]:
# Extract the number of coordinate pairs
coords = range(len(building_centre_coords))

# Accessibility for adults
# For a set of amenities, calculate how many are within a 1,200 metres range from the selected population weighted centroid.
schools = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","school"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        schools.append(amenity_gdf.shape[0])
        
        
supermarkets = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#shop","supermarket"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        supermarkets.append(amenity_gdf.shape[0])
        

pharmacies = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","pharmacy"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        pharmacies.append(amenity_gdf.shape[0])


atms = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","atm"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        atms.append(amenity_gdf.shape[0])


parks = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","park"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        parks.append(amenity_gdf.shape[0])


cinemas = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","cinema"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        cinemas.append(amenity_gdf.shape[0])


gyms = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","gym"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        gyms.append(amenity_gdf.shape[0])

index_adults = pd.DataFrame({'buildings_id':buildings_gdf.buildings_id,'schools':schools, 'supermarkets':supermarkets, 'pharmacies':pharmacies, 'atms':atms, 'parks':parks, 'cinemas':cinemas, 'gyms':gyms})


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Stream removed", grpc_status:2, created_time:"2023-04-27T09:31:42.710383812+00:00"}"
>

In [ ]:
# Accessibility for seniors
# For a set of amenities, calculate how many are within a 800 metres range from the selected population weighted centroid.
schools = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","school"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        schools.append(amenity_gdf.shape[0])
        
        
supermarkets = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#shop","supermarket"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        supermarkets.append(amenity_gdf.shape[0])
        

pharmacies = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","pharmacy"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        pharmacies.append(amenity_gdf.shape[0])


atms = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","atm"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        atms.append(amenity_gdf.shape[0])


parks = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","park"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        parks.append(amenity_gdf.shape[0])


cinemas = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","cinema"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        cinemas.append(amenity_gdf.shape[0])


gyms = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","gym"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        gyms.append(amenity_gdf.shape[0])

index_seniors = pd.DataFrame({'buildings_id':buildings_gdf.buildings_id, 'schools':schools, 'supermarkets':supermarkets, 'pharmacies':pharmacies, 'atms':atms, 'parks':parks, 'cinemas':cinemas, 'gyms':gyms})


In [ ]:
# Accessibility for children
# For a set of amenities, calculate how many are within a 500 metres range from the selected population weighted centroid.
schools = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","school"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        schools.append(amenity_gdf.shape[0])

parks = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","park"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        parks.append(amenity_gdf.shape[0])


playgrounds = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","playground"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        playgrounds.append(amenity_gdf.shape[0])

pitches = []
for i in coords:
        point_x = building_centre_coords[i][1]
        point_y = building_centre_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","pitch"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        pitches.append(amenity_gdf.shape[0])

index_children = pd.DataFrame({'buildings_id':buildings_gdf.buildings_id,'schools':schools, 'parks':parks, 'playgrounds':playgrounds, 'pitches':pitches})

In [ ]:
# Calculate accessibility score adults
index_adults['score'] = index_adults[['schools','supermarkets','pharmacies', 'atms','parks','cinemas','gyms']].astype(bool).sum(axis=1)
index_adults['score_norm'] = (index_adults['score'] - index_adults['score'].min())/ (index_adults['score'].max() - index_adults['score'].min())

# Calculate accessibility score seniors
index_seniors['score'] = index_seniors[['schools','supermarkets','pharmacies', 'atms','parks','cinemas','gyms']].astype(bool).sum(axis=1)
index_seniors['score_norm'] = (index_seniors['score'] - index_seniors['score'].min())/ (index_seniors['score'].max() - index_adults['score'].min())

# Calculate accessibility score children
index_children['score'] = index_children[['schools','parks','playgrounds','pitches']].astype(bool).sum(axis=1)
index_children['score_norm'] = (index_children['score'] - index_children['score'].min())/ (index_children['score'].max() - index_children['score'].min())

In [ ]:
# Attach LSOA geometry

index_adults = index_adults.reset_index().merge(buildings_gdf[['buildings_id','geometry']], on = 'buildings_id')
index_seniors = index_seniors.reset_index().merge(buildings_gdf[['buildings_id','geometry']], on = 'buildings_id')
index_children = index_children.reset_index().merge(buildings_gdf[['buildings_id','geometry']], on = 'buildings_id')

In [ ]:
gpd.GeoDataFrame(index_adults).to_file(data_directory + '/index_adults_buildings.geojson')
gpd.GeoDataFrame(index_seniors).to_file(data_directory + '/index_seniors_buildings.geojson')
gpd.GeoDataFrame(index_children).to_file(data_directory + '/index_children_buildings.geojson')